<a href="https://colab.research.google.com/github/Hamoda-dabbit/XLNet/blob/main/xlnettweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

import data

you can download dataset from link:

https://drive.google.com/drive/folders/1bCv8rkmkMem5hkpXYmB6XYRmMIlGtp83?usp=sharing

In [1]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string

train_df = pd.read_csv("/content/drive/MyDrive/ماجستير/ذكاء/dataset/tweet/train.csv")
train_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [2]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

------------------
انشاء مصفوفة بيانات جديدة تحوي عمودي النص والصنف فقط وتسمية العمودين

text,label

In [3]:
train_df = pd.DataFrame({
    'text': train_df['text'].replace(r'\n', ' ', regex=True),
    'label':train_df['target']
})
train_df

,text,label
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


------------
تجهيز النصوص(تنظيف النصوص)

In [4]:
def clean(df):
  df = shuffle(df)
  print(np.sum(df.isnull().any(axis=1)))

  clean_df= df.dropna()
  print(np.sum(clean_df.isnull().any(axis=1)))
  #-------------
  print (df.notnull().any(axis = 0))
  print(len(df))

  #-------------
  english_punctuations = string.punctuation
  punctuations_list =  english_punctuations

  def remove_punctuations(text):
      translator = str.maketrans('', '', punctuations_list)
      return text.translate(translator)
  #-------------    
  def remove_repeating_char(text):
      return re.sub(r'(.)\1+', r'\1', text)
  #-------------
  def processDocument(doc, stemmer): 

      #Replace @username with empty string
      doc = re.sub(r'@[^\s]+', ' ', doc)
      doc = re.sub(r'_', ' ', doc)
      doc = re.sub(r'\n', ' ', doc)
      doc = re.sub(r'\d', '', doc)
      #Convert www.* or https?://* to " "
      doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
      #Replace #word with word
      doc = re.sub(r'#([^\s]+)', r'\1', doc)
      doc = doc.lower()
      # remove punctuations
      doc= remove_punctuations(doc)
      # remove repeated letters
      doc=remove_repeating_char(doc)
      #stemming
      doc = stemmer.stem(doc)
      return doc

  stemmer = ISRIStemmer()
  clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
  print(clean_df)
  return clean_df

In [5]:
train_df=clean(train_df)

0
0
text     True
label    True
dtype: bool
7613
                                                   text  label
3049  usgs eq m km e of anchorage alaska time utc ea...      1
6243  ûï everyone here ah i hate snow me lol u cal ...      1
3064  earthquake m km n of brenas puerto rico time u...      1
3313   and from what i can tel they responded to tod...      1
7601  breaking la refugio oil spil may have ben cost...      1
...                                                 ...    ...
3208  sustainability task force presents levy to fun...      1
3015             curently driving through a dust storm       1
5277  la times refugio oil spil may have ben costlie...      1
742                  my ears are bleding i hate stefano      0
2074                       is ros realy dead askcharley      0

[7613 rows x 2 columns]


----------
تقسيم البيانات لتدريب واختبار

In [6]:

eval_df=train_df[5000:]
train_df=train_df[:5000]
eval_df

,text,label
535,chiason sens cant come to deal coloradoavalanc...,1
3032,earthquake in sicily italy iphone users downl...,1
5583,the latest more homes razed by northern califo...,1
1666,and when those waves shal riple colide its on ...,0
3242,men escape car engulfed in flames in parleys c...,1
...,...,...
3208,sustainability task force presents levy to fun...,1
3015,curently driving through a dust storm,1
5277,la times refugio oil spil may have ben costlie...,1
742,my ears are bleding i hate stefano,0


-------------
انشاء موديل 

 XLNet base

In [ ]:
from simpletransformers.classification import ClassificationModel,ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs=2
model_args.max_seq_length=256
model_args.overwrite_output_dir=True
model = ClassificationModel('xlnet', 'xlnet-base-cased',args=model_args,use_cuda=1)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

----------
 تدريب الموديل

In [ ]:
import sklearn
# Train the model
model.train_model(train_df,acc=sklearn.metrics.accuracy_score)


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/625 [00:00<?, ?it/s]

(625, 0.5175529132843018)

-------------
اختبار الموديل

In [ ]:
import sklearn
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/2613 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/327 [00:00<?, ?it/s]

--------------
النتائج

In [ ]:
result

{'mcc': 0.6120842252221014,
 'tp': 836,
 'tn': 1283,
 'fp': 207,
 'fn': 287,
 'auroc': 0.8694364926162543,
 'auprc': 0.8578364831747376,
 'acc': 0.8109452736318408,
 'eval_loss': 0.4748990411787587}

In [ ]:
eval_df.label.value_counts()

0    1490
1    1123
Name: label, dtype: int64